In [1]:
import sys
import numpy as np

from collections import defaultdict

# Configuration

In [2]:
USE_APPEND_MYMODULES = True

if USE_APPEND_MYMODULES is True:
    sys.path.append('./module/')

In [3]:
# my module

from corpus_class import Corpus

In [4]:
corpus_fname = "./data/2016-10-28_article_all_normed.txt"

max_l_length = 10
min_count = 30

# Class

In [5]:
corpus = Corpus(corpus_fname, iter_sent=False)

In [6]:
corpus.iter_sent = True

L = defaultdict(lambda: 0)

for num_sent, sent in enumerate(corpus):
    if num_sent % 5000 == 0:
        sys.stdout.write('\r inserting %d sents... ' % num_sent)
    for token in sent.split():
        for e in range(1, min(max_l_length, len(token)) + 1):
            subword = token[:e]
            L[subword] += 1
print('\rinserting subwords into L: done')
print('num subword = %d' % len(L))

L = {subword:freq for subword, freq in L.items() if freq >= min_count}
print('num subword = %d (after pruning with min count %d)' % (len(L), min_count))

inserting subwords into L: done
num subword = 578867
num subword = 30922 (after pruning with min count 30)


In [7]:
len(corpus_fname)

40

In [8]:
def get_cohesion(word):
    
    # 글자가 아니거나 공백, word가 L에 없는 경우 (빈도수가 min_count 이하이거나 아예 코퍼스에 등장하지 않았던 경우)
    if (not word) or ((word in L) == False):
        return 0.0
    
    if len(word) == 1:
        return 1.0
    
    word_freq = L.get(word, 0)
    base_freq = L.get(word[:1], 0)
    
    if base_freq == 0:
        return 0.0
    else:
        return np.power(word_freq/base_freq, 1 / (len(word) - 1))

In [9]:
print('청와대 : ', get_cohesion('청와대'))

청와대 :  0.722212060689


- 길이가 2 이상인 subwords에 대하여 cohesion을 미리 계산
- 앞에서 min_count로 필터링을 한 번 했기 때문에 적은 개수의 subwords만 cohesion을 계산

In [10]:
cohesion_dict = {word:get_cohesion(word) for word in L if len(word)>=2}
len(cohesion_dict)

29908

In [11]:
for word in ['청와', '청와대', '청와대는', '민정수석', '민정수석이', '박근', '박근혜', '박근혜의']:
    print('%s = %.3f\n' % (word, cohesion_dict.get(word, 0.0)))

청와 = 0.522

청와대 = 0.722

청와대는 = 0.283

민정수석 = 0.474

민정수석이 = 0.313

박근 = 0.294

박근혜 = 0.539

박근혜의 = 0.144



어절 word가 입력되었을 때, L들에서 cohesion이 가장 높은 subword를 잘라내는 토크나이저를 만들어 봅시다.
길이가 2 이상일 때 cohesion이 정의되기 때문에 길이가 2 이하인 단어는 그대로 return 합니다.
subword 의 ending point e는 길이가 2부터 'word의 길이 혹은 L의 최대 길이'의 min까지 입니다. range(b,e)에서 e는 포함되지 않기 때문에 +1을 해주는 것을 잊지 맙시다.

### L_tokenize

In [12]:
def L_tokenize(word, tolerance=0.1):
    """
     L들에서 cohesion이 가장 높은 subword를 잘라내는 토크나이저
     길이가 2 이상일 때 cohesion이 정의되기 때문에 길이가 2 이하인 단어는 그대로 return
     subword 의 ending point e는 길이가 2부터 'word의 길이 혹은 L의 최대 길이'의 min
     
     parameter
     tolerance: If this is True, return value when max cohesion score is lager than tolerance
    """
    
    if len(word) <= 2:
        return word
    
    score = []
    for e in range(2, min(max_l_length, len(word))+1):
        subword = word[:e]
        score.append((subword, cohesion_dict.get(subword, 0.0), e)) # (word, cohesion, length)
    
    return sorted(score, key=lambda x:(x[1], x[2]), reverse=True)[0]

In [13]:
for word in ['청와대', '청와대는', '민정수석', '민정수석이', '박근혜', '박근혜의']:
    print('%s --> %s\n' % (word, L_tokenize(word)))

청와대 --> ('청와대', 0.7222120606891772, 3)

청와대는 --> ('청와대', 0.7222120606891772, 3)

민정수석 --> ('민정수석', 0.47361601469726433, 4)

민정수석이 --> ('민정수석', 0.47361601469726433, 4)

박근혜 --> ('박근혜', 0.5386349787638236, 3)

박근혜의 --> ('박근혜', 0.5386349787638236, 3)



### L_tokenize_w_tolerance
- L_tokenizer는 상황에 맞게 튜닝
- Cohesion의 최대값을 지니는 subword를 선택하는 것이 아니라, 최대값과의 크기가 0.1 이하로 차이가 나는 subword 중에서는 가장 긴 subword를 추출

In [14]:
my_cohesion_dict = {
    '기자': 0.5,
    '기자단': 0.45
}

def L_tokenize_w_tolerance(word, tolerance=0.1):
    """
    parameter: tolerance
    Cohesion의 최대값을 지니는 subword를 선택하는 것이 아니라, 
    최대값과의 크기가 tolerance 값 이하로 차이가 나는 subword 중에서 가장 긴 subword를 추출
    """
    if len(word) <= 2:
        return word
    
    score = []
    for e in range(2, min(len(word), max_l_length)+1):
        subword = word[:e]
        score.append((subword, my_cohesion_dict.get(subword, 0), e)) # (word, cohesion, length)
    
    max_score = max([s[1] for s in score])
    score = [s for s in score if (max_score - s[1]) <= tolerance]
    
    return sorted(score, key=lambda x:x[2], reverse=True)[0][0]


for word in ['기자단에']:
    print('w tolerance: %s --> %s' % (word, L_tokenize(word)))
    print('wo tolerance: %s --> %s\n' % (word, L_tokenize_w_tolerance(word)))

w tolerance: 기자단에 --> ('기자', 0.36924713107020946, 2)
wo tolerance: 기자단에 --> 기자단

